In [1]:
import arcpy
import sqlite3
import pandas as pd
import os
import arcgis

In [ ]:
# this is the folder that this notebook is in. 
# you could change this if you want to do this work elsewhere
folder = os.path.abspath('')
folder

In [3]:
# create an sqlite geodatabase with ST Geometry type
sl_db = arcpy.management.CreateSQLiteDatabase(
    out_database_name = os.path.join(folder, 'data'),
    spatial_type = "ST_GEOMETRY"
)

In [4]:
# extract the path from the Result object
sl_db_path = sl_db[0]

In [ ]:
# path to the source data in the file geodatabase
source_fc = os.path.join(folder, 'Counties.gdb', 'Counties')

# path to the destination data in the sqlite geodatabase
destination_fc = os.path.join(sl_db_path, "Counties")

# export the data
fc_counties = arcpy.conversion.ExportFeatures(
    source_fc,
    destination_fc
)

In [ ]:
# walk the sqlite database and print data table names
for root, dirs, files in arcpy.da.Walk(sl_db):
    for dir in dirs:
        print(dir)
    for file in files:
        print(file)

In [ ]:
# set up a direct connection to the sqlite database
sl_conn = sqlite3.connect(sl_db_path)
sl_conn

In [8]:
# create a cursor
sl_cursor = sl_conn.cursor()

In [ ]:
# use the cursor to execute SQL statement
sl_cursor.execute("select * from sqlite_master")

In [10]:
# retrieve the results
results = sl_cursor.fetchall()

In [ ]:
print(f"results type: {type(results)}")
print(f"length of results: {len(results)}")
print(f"first item type: {type(results[0])}")
print(f"first item length: {len(results[0])}")

In [ ]:
results[0]

In [ ]:
# use pandas to read an sql query
df_db_objects = pd.read_sql("select * from sqlite_master", sl_conn)
df_db_objects

In [ ]:
# path to csv file
csv_path = r".\block_groups.csv"

# read the csv
df_block_groups = pd.read_csv(csv_path, dtype={'GEOID':str})
df_block_groups

In [ ]:
# trim the GEOID to five characters (county GEOID)
df_block_groups['GEOID'].str[:5]

In [16]:
# create a new column representing the county GEOID
df_block_groups['county_geoid'] = df_block_groups['GEOID'].str[:5]

In [ ]:
# write the census block dataframe into the database
df_block_groups.to_sql(
    name = "BlockGroups", 
    con = sl_conn, 
    index = False
)

In [18]:
# SQL statement for summarizing the population and housing units by county
sql = """
SELECT
    county_geoid,
    SUM(Population) as county_population,
    SUM(Housing_Units) as county_housing_units
FROM
    BlockGroups
GROUP BY
    county_geoid

"""

In [ ]:
pd.read_sql(sql, sl_conn).head()

In [ ]:
# create a view from the previous select statement
sql_summary_view = """
CREATE VIEW CountyStats AS
SELECT
    county_geoid,
    SUM(Population) as county_population,
    SUM(Housing_Units) as county_housing_units
FROM
    BlockGroups
GROUP BY
    county_geoid

"""
sl_cursor.execute(sql_summary_view)

In [ ]:
# read the data from the view
pd.read_sql("select * from CountyStats", sl_conn).head()

In [22]:
# SQL statement creating a joined table with the spatial and non spatial data
sql_join = """
CREATE VIEW CountyStatsSpatial AS
SELECT
    fc.OBJECTID,
    fc.name,
    tbl.county_population,
    tbl.county_housing_units,
    fc.shape
FROM
    Counties fc
    JOIN
        CountyStats tbl
    ON
        fc.geoid = tbl.county_geoid
"""

In [ ]:
# use the cursor to execute the SQL statement
sl_cursor.execute(sql_join)

In [ ]:
# read the data from the view to validate
pd.read_sql("select * from CountyStatsSpatial", sl_conn).head()

In [ ]:
# use pandas and the arcgis spatial accessor to read spatial view
df_out = pd.DataFrame.spatial.from_featureclass(
    os.path.join(sl_db[0],"main.CountyStatsSpatial")
)
df_out.head()

In [ ]:
# create a web map
web_map = arcgis.GIS().map("California")

# add the data
df_out.spatial.plot(map_widget=web_map)

# display the web map
web_map

In [28]:
# get the renderer for the county layer
renderer_manager = web_map.content.renderer(0)

# use smart mapping
smart_mapper = renderer_manager.smart_mapping()

# create a class breaks renderer
smart_mapper.class_breaks_renderer(
    break_type = 'color',
    field = 'county_housing_units',
    classification_method = 'natural-breaks',
    num_classes = 10
)

Copyright 2025 Esri